In [1]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import Audio, Dataset

torch.set_float32_matmul_precision("high")

/home/demontego/whisper/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

In [3]:
model_id = "openai/whisper-large-v3-turbo"
model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True
).to(device)

In [4]:
model.generation_config.cache_implementation = "static"
model.generation_config.max_new_tokens = 256
model.forward = torch.compile(model.forward, mode="reduce-overhead", fullgraph=True)

In [10]:
processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device,
)

Device set to use cuda:0


In [11]:
audio = Audio(sampling_rate=16_000, mono=True, decode=True)
audio_dataset = Dataset.from_dict({"audio": [
    "/home/demontego/whisper/tests/test_data/output.wav",
    "/home/demontego/whisper/tests/test_data/whisper_audio.wav",
    ]}).cast_column("audio", audio)
sample = audio_dataset[1]['audio']

In [12]:
from torch.nn.attention import SDPBackend, sdpa_kernel
from tqdm import tqdm

for _ in tqdm(range(2), desc="Warm-up step"):
    with sdpa_kernel(SDPBackend.MATH):
        result = pipe(sample.copy(), generate_kwargs={"min_new_tokens": 256, "max_new_tokens": 256}, return_timestamps=True)


Warm-up step: 100%|██████████| 2/2 [00:09<00:00,  4.60s/it]


In [16]:


with sdpa_kernel(SDPBackend.MATH):
    result = pipe(audio_dataset['audio'].copy(), return_timestamps=True)

/home/demontego/whisper/.venv/lib/python3.13/site-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.


In [18]:
result[1]

{'text': ' Продукт-менеджер, у него огромный опыт работы с GNI, с разными продуктами, с бизнесом, понимает ограничения, в том числе запросы, поэтому он здесь очень полезен. Наверное, если сделаешь небольшое интро про то, где работал, тоже, чтобы ребятам было комфортно, и потом расскажу о ребятах. Да, да, да. Последний год я работаю над консалтингом в AI. Вот мы со Сбером, с Яндексом, вот с Нитологией и с другими прикольными компаниями как раз повнедряли какие-то разные штуки.',
 'chunks': [{'timestamp': (0.32, 5.2),
   'text': ' Продукт-менеджер, у него огромный опыт работы с GNI, с разными продуктами, с бизнесом,'},
  {'timestamp': (5.3, 9.7),
   'text': ' понимает ограничения, в том числе запросы, поэтому он здесь очень полезен.'},
  {'timestamp': (10.82, 16.0),
   'text': ' Наверное, если сделаешь небольшое интро про то, где работал, тоже, чтобы ребятам было комфортно,'},
  {'timestamp': (16.12, 17.28), 'text': ' и потом расскажу о ребятах.'},
  {'timestamp': (17.32, 21.56),
   'tex

In [13]:
sample2 = audio_dataset['audio']

for _ in tqdm(range(30), desc="Warm-up step"):
    with sdpa_kernel(SDPBackend.MATH):
        result = pipe(sample2.copy(), return_timestamps=True)

Warm-up step:  17%|█▋        | 5/30 [00:46<03:50,  9.22s/it]


KeyboardInterrupt: 

In [9]:
result

{'text': ' я читаю про дипсик и просто как индекс филе до этого не додумались просто нам прикольная прям архитектура что типа на поверхность еще там способ обучения типа он на много раз обсуждался вот почему-то и редко и видео в применении я знаю почему они пошли по этому пути они начали обучать давно очень модель кодинга дипсик кодер у них была лучшая модель вреди всех даже лучше чем чат gpt и они такие надо еще лучше и расширили за пределы кодинга свою модель гениев считаю отчетом вот такое reinforcement learning как-то используется активно или в чем-то да там rl активно используется и там четыре этапа обучения идет вот номер эль там типа решает и первый второй этап он реальный и типа нет модели вознаграждения есть только правильный неправильный ответ вот типа всякие математические задачи логические задачи по химии по биологии по физике вот что-то такое это из науки на финальной стадии у них типа нет на финальной стадии у них есть типа ответ от модели и его сравнивают с истинным наск

In [1]:
import librosa

In [2]:
w, s = librosa.load("/home/demontego/whisper/tests/test_data/output.wav")

In [3]:
import numpy as np
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor
from transformers.pipelines.automatic_speech_recognition import AutomaticSpeechRecognitionPipeline
from torch.nn.attention import SDPBackend, sdpa_kernel
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline

/home/demontego/whisper/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
seconds_per_chunk = 30
device = torch.device('cuda')
model_id = "openai/whisper-large-v3-turbo"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    pretrained_model_name_or_path=model_id,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    attn_implementation='sdpa',
)
model.to(device)

generate_kwargs = {
    'max_new_tokens': 444,
    'num_beams': 3,
    'condition_on_prev_tokens': False,
    'compression_ratio_threshold': 1.35,
    'temperature': (0.0, 0.2, 0.4, 0.6, 0.8, 1.0),
    'logprob_threshold': -1.0,
}

processor = AutoProcessor.from_pretrained(model_id)

pipe = AutomaticSpeechRecognitionPipeline(
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch.float16,
    device=device,
    # chunk_length_s=seconds_per_chunk,  # Add chunking here
    # batch_size=16,  # Add batch size here
)

Device set to use cuda


In [19]:
result = pipe(w, generate_kwargs=generate_kwargs, return_timestamps=True)
from pprint import pprint
pprint(result)

/home/demontego/whisper/.venv/lib/python3.13/site-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


{'chunks': [{'text': ' Субтитры создавал DimaTorzok',
             'timestamp': (0.0, 29.98)},
            {'text': ' Я читаю про дипсик и просто...',
             'timestamp': (0.0, 4.0)},
            {'text': ' Как индексели до этого не додумались, просто...',
             'timestamp': (6.0, 10.0)},
            {'text': ' А, нам прикольная прям архитектура, что типа на '
                     'поверхности что-то лежало.',
             'timestamp': (12.0, 18.0)},
            {'text': ' Да.', 'timestamp': (19.0, 20.0)},
            {'text': ' Там способ обучения, типа, он...',
             'timestamp': (20.0, 23.0)},
            {'text': '', 'timestamp': (27.0, 0.0)},
            {'text': ' И на много раз обсуждался Но почему-то я редко видел '
                     'его в применении',
             'timestamp': (6.86, 13.76)},
            {'text': ' А я знаю, почему они пошли по этому пути',
             'timestamp': (13.76, 24.46)},
            {'text': '', 'timestamp': (24.46, 0.0)},
 